In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm, trange
from tabulate import tabulate
from datetime import datetime
import time
 
from sklearn.model_selection import train_test_split
from sklearn import metrics
 
import cv2
import gc
import os 

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')

dataset_path = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets/'
classes = os.listdir(dataset_path)
print(f'classes: {classes}')

classes: ['lung_aca', 'lung_scc', 'lung_n']


In [13]:
IMAGE_RESOLUTION = (256, 256)
BATCH_SIZE = 64
RANDOM_SEED = 42
VALID_SPLIT = 0.2
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize(IMAGE_RESOLUTION),  # Resize images
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=means, std=stds)  # Normalize
])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

generator = torch.Generator().manual_seed(RANDOM_SEED)
train_set, val_set = torch.utils.data.random_split(dataset, [1-VALID_SPLIT, VALID_SPLIT], generator=generator)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

img, label = dataset[10]
print(f'Image Size: {img.shape}')
print(f'Training Set: {len(train_set)}, Valid Set: {len(val_set)}, Number of Batches: {len(train_loader)}')

Image Size: torch.Size([3, 256, 256])
Training Set: 12000, Valid Set: 3000, Number of Batches: 188


In [14]:
HIDDEN_LAYERS = 3
LEARNING_RATE = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using device: {device}')

# Creating a CNN class
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.act1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.act2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.act3 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(32 * IMAGE_RESOLUTION[0]//8 * IMAGE_RESOLUTION[1]//8, 128)
        self.actfc1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(128, 3)
    def forward(self, x):
        x = self.pool1(self.act1(self.bn1(self.conv1(x))))
        x = self.pool2(self.act2(self.bn2(self.conv2(x))))
        x = self.pool3(self.act3(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.actfc1(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = Net()
model = nn.DataParallel(model)
model = model.to(device=device)

# Set optimizer with optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

using device: cuda


In [15]:
def train_network(n_epochs, optimizer, model, train_loader, valid_loader, loss_fn = nn.CrossEntropyLoss()):
    pbar = trange(n_epochs, desc='Epoch')
    loss_train_array = []
    loss_valid_array = []
    start_time = time.time()
    for i, epoch in enumerate(pbar):
        loss_train = 0.0
        model.train()
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            model = model.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        
        loss_valid = 0.0
        model.eval()
        
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in valid_loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                model = model.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0] 
                correct += int((predicted == labels).sum())
                loss = loss_fn(outputs, labels)
                loss_valid += loss.item()
            
        pbar.set_postfix({'train loss': loss_train/len(train_loader), 'valid loss': loss_valid/len(valid_loader), 'valid acc': f'{correct/total*100}%'}, refresh=True)
        print('|-------------------------------------------------------------------------------------------|')
        loss_train_array.append(loss_train/len(train_loader))
        loss_valid_array.append(loss_valid/len(valid_loader))
        
    end_time = time.time()
    model.eval()
    accuracies = []
    for name, loader in [("Training", train_loader), ("Validation", valid_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                model.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
            print(f'{name} Accuracy: {correct/total*100:.1f}')
            accuracies.append(correct/total*100)
        
    return loss_train_array, loss_valid_array, accuracies, (end_time - start_time)
        

In [16]:
loss_train, loss_valid, accuracies, times = train_network(20, optimizer, model, train_loader, val_loader)

Epoch:   5%|▌         | 1/20 [01:14<23:40, 74.78s/it, train loss=0.51, valid loss=0.224, valid acc=91.53333333333333%]

|-------------------------------------------------------------------------------------------|


Epoch:  10%|█         | 2/20 [02:29<22:23, 74.65s/it, train loss=0.172, valid loss=0.172, valid acc=93.36666666666666%]

|-------------------------------------------------------------------------------------------|


Epoch:  15%|█▌        | 3/20 [03:45<21:18, 75.22s/it, train loss=0.132, valid loss=2.39, valid acc=67.26666666666667%] 

|-------------------------------------------------------------------------------------------|


Epoch:  20%|██        | 4/20 [05:00<20:02, 75.13s/it, train loss=0.208, valid loss=0.207, valid acc=92.13333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch:  25%|██▌       | 5/20 [06:14<18:44, 74.97s/it, train loss=0.11, valid loss=0.147, valid acc=94.86666666666666%] 

|-------------------------------------------------------------------------------------------|


Epoch:  30%|███       | 6/20 [07:29<17:26, 74.73s/it, train loss=0.0625, valid loss=0.135, valid acc=95.6%]           

|-------------------------------------------------------------------------------------------|


Epoch:  35%|███▌      | 7/20 [08:42<16:04, 74.22s/it, train loss=0.0536, valid loss=0.135, valid acc=95.8%]

|-------------------------------------------------------------------------------------------|


Epoch:  40%|████      | 8/20 [09:56<14:49, 74.15s/it, train loss=0.0351, valid loss=0.127, valid acc=95.83333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch:  45%|████▌     | 9/20 [11:10<13:34, 74.03s/it, train loss=0.0243, valid loss=0.143, valid acc=96.1%]             

|-------------------------------------------------------------------------------------------|


Epoch:  50%|█████     | 10/20 [12:23<12:19, 73.91s/it, train loss=0.0227, valid loss=0.148, valid acc=95.93333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch:  55%|█████▌    | 11/20 [13:37<11:04, 73.81s/it, train loss=0.0182, valid loss=0.177, valid acc=95.66666666666667%]

|-------------------------------------------------------------------------------------------|


Epoch:  60%|██████    | 12/20 [14:50<09:49, 73.74s/it, train loss=0.0155, valid loss=0.166, valid acc=95.83333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch:  65%|██████▌   | 13/20 [16:04<08:34, 73.56s/it, train loss=0.00641, valid loss=0.172, valid acc=96.0%]            

|-------------------------------------------------------------------------------------------|


Epoch:  70%|███████   | 14/20 [17:17<07:21, 73.65s/it, train loss=0.00971, valid loss=0.261, valid acc=94.13333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch:  75%|███████▌  | 15/20 [18:31<06:08, 73.61s/it, train loss=0.0225, valid loss=0.19, valid acc=95.33333333333334%]  

|-------------------------------------------------------------------------------------------|


Epoch:  80%|████████  | 16/20 [19:46<04:55, 73.98s/it, train loss=0.0231, valid loss=0.167, valid acc=95.63333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch:  85%|████████▌ | 17/20 [21:01<03:43, 74.40s/it, train loss=0.0223, valid loss=0.154, valid acc=95.66666666666667%]

|-------------------------------------------------------------------------------------------|


Epoch:  90%|█████████ | 18/20 [22:16<02:28, 74.45s/it, train loss=0.013, valid loss=0.172, valid acc=96.26666666666667%] 

|-------------------------------------------------------------------------------------------|


Epoch:  95%|█████████▌| 19/20 [23:28<01:13, 73.90s/it, train loss=0.00994, valid loss=0.195, valid acc=95.63333333333334%]

|-------------------------------------------------------------------------------------------|


Epoch: 100%|██████████| 20/20 [24:41<00:00, 74.06s/it, train loss=0.00773, valid loss=0.186, valid acc=96.33333333333334%]

|-------------------------------------------------------------------------------------------|


Training Accuracy: 100.0
Validation Accuracy: 96.3


Text(0, 0.5, 'Loss')

In [17]:
fig, ax = plt.subplots()
ax.plot(range(20), loss_train, label='training')
ax.plot(range(20), loss_valid, label='validation')
ax.legend()
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")

In [18]:
print(f'{times/60} minutes to train for 20 epochs')
print(f'training accuracy: {accuracies[0]}%')
print(f'validation accuracy: {accuracies[1]}%')

24.688261433442435 minutes to train for 20 epochs
training accuracy: 100.0%
validation accuracy: 96.33333333333334%
